# 🛒 Taller de SQL - Caso Olist E-commerce

## Contexto del Negocio

**Olist** es una plataforma de e-commerce brasileña que conecta pequeños y medianos comerciantes con clientes en todo Brasil. Funciona como un marketplace donde los vendedores publican sus productos y los clientes pueden comprarlos online.

El dataset contiene información de **~100,000 pedidos** realizados entre 2016 y 2018.

### Tu rol
Eres un **Analista de Datos** en Olist. El equipo de negocio necesita respuestas a preguntas clave sobre ventas, clientes y productos para tomar decisiones estratégicas.

---

## 📊 Estructura del Dataset

El dataset tiene **9 tablas** relacionadas:

```
┌─────────────────┐      ┌──────────────────┐      ┌─────────────────┐
│   customers     │      │     orders       │      │    sellers      │
│─────────────────│      │──────────────────│      │─────────────────│
│ customer_id (PK)│◄────►│ order_id (PK)    │      │ seller_id (PK)  │
│ city            │      │ customer_id (FK) │      │ city            │
│ state           │      │ order_status     │      │ state           │
└─────────────────┘      │ purchase_date    │      └────────┬────────┘
                         │ delivered_date   │               │
                         └────────┬─────────┘               │
                                  │                         │
                         ┌────────▼─────────┐               │
                         │   order_items    │◄──────────────┘
                         │──────────────────│
                         │ order_id (FK)    │      ┌─────────────────┐
                         │ product_id (FK)  │◄────►│    products     │
                         │ seller_id (FK)   │      │─────────────────│
                         │ price            │      │ product_id (PK) │
                         │ freight_value    │      │ category_name   │
                         └────────┬─────────┘      └─────────────────┘
                                  │
                         ┌────────▼─────────┐      ┌─────────────────┐
                         │  order_payments  │      │  order_reviews  │
                         │──────────────────│      │─────────────────│
                         │ order_id (FK)    │      │ order_id (FK)   │
                         │ payment_type     │      │ review_score    │
                         │ payment_value    │      │ review_comment  │
                         └──────────────────┘      └─────────────────┘
```

---

## ⚙️ Configuración Inicial

Primero, descargamos el dataset y configuramos la base de datos SQLite.

In [ ]:
# Instalación de dependencias
!pip install pandas sqlalchemy kaggle --quiet

In [ ]:
# Descarga del dataset de Kaggle
# NOTA: Necesitas configurar tu API key de Kaggle
# 1. Ve a kaggle.com -> Account -> Create New API Token
# 2. Coloca el archivo kaggle.json en ~/.kaggle/

!kaggle datasets download -d olistbr/brazilian-ecommerce --unzip -p ./data/

In [ ]:
# Alternativa: Si no tienes Kaggle configurado, 
# descarga manualmente desde: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
# y descomprime en la carpeta ./data/

import os
import pandas as pd
import sqlite3

# Verificar que los archivos existen
data_path = './data/'
archivos = os.listdir(data_path) if os.path.exists(data_path) else []
print("Archivos disponibles:")
for f in archivos:
    print(f"  - {f}")

In [ ]:
# Crear base de datos SQLite con los datos de Olist

def crear_base_datos():
    """Carga todos los CSVs en una base de datos SQLite"""
    
    conn = sqlite3.connect('olist.db')
    
    # Mapeo de archivos a nombres de tablas (más cortos y amigables)
    tablas = {
        'olist_customers_dataset.csv': 'customers',
        'olist_orders_dataset.csv': 'orders',
        'olist_order_items_dataset.csv': 'order_items',
        'olist_order_payments_dataset.csv': 'order_payments',
        'olist_order_reviews_dataset.csv': 'order_reviews',
        'olist_products_dataset.csv': 'products',
        'olist_sellers_dataset.csv': 'sellers',
        'olist_geolocation_dataset.csv': 'geolocation',
        'product_category_name_translation.csv': 'category_translation'
    }
    
    for archivo, nombre_tabla in tablas.items():
        ruta = os.path.join(data_path, archivo)
        if os.path.exists(ruta):
            df = pd.read_csv(ruta)
            df.to_sql(nombre_tabla, conn, if_exists='replace', index=False)
            print(f"✓ Tabla '{nombre_tabla}' creada con {len(df):,} registros")
        else:
            print(f"✗ Archivo no encontrado: {archivo}")
    
    conn.close()
    print("\n✅ Base de datos 'olist.db' creada exitosamente!")

crear_base_datos()

In [ ]:
# Configurar conexión para ejecutar SQL directamente en las celdas

%load_ext sql
%sql sqlite:///olist.db

print("✅ Conexión establecida. Puedes usar %%sql para ejecutar consultas.")

---

## 🔹 PARTE 1: Exploración Básica (SELECT, WHERE, ORDER BY)

### Objetivo de Negocio
Antes de hacer análisis complejos, necesitamos **entender los datos** que tenemos disponibles.

### Ejercicio 1.1 - Conocer las tablas

**Pregunta de negocio:** ¿Qué información tenemos disponible en la base de datos?

Ejecuta la siguiente consulta para ver las tablas disponibles:

In [ ]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

### Ejercicio 1.2 - Explorar la tabla de pedidos

**Pregunta de negocio:** ¿Cómo se ve un pedido en nuestro sistema?

Muestra los primeros 5 registros de la tabla `orders`:

In [ ]:
%%sql
-- Tu código aquí
SELECT * FROM orders LIMIT 5;

### Ejercicio 1.3 - Estados de los pedidos

**Pregunta de negocio:** ¿Qué estados puede tener un pedido?

Lista los valores únicos de `order_status`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT DISTINCT order_status 
FROM orders;
```
</details>

### Ejercicio 1.4 - Filtrar pedidos entregados

**Pregunta de negocio:** ¿Cuántos pedidos han sido entregados exitosamente?

Cuenta los pedidos con `order_status = 'delivered'`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT COUNT(*) as pedidos_entregados
FROM orders
WHERE order_status = 'delivered';
```
</details>

### Ejercicio 1.5 - Productos más caros

**Pregunta de negocio:** ¿Cuáles son los 10 productos más caros vendidos?

Consulta la tabla `order_items` y ordena por `price` descendente:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT order_id, product_id, price
FROM order_items
ORDER BY price DESC
LIMIT 10;
```
</details>

---

## 🔹 PARTE 2: Agregaciones (COUNT, SUM, AVG, GROUP BY)

### Objetivo de Negocio
Ahora que conocemos los datos, vamos a **resumir y agregar** información para obtener métricas clave.

### Ejercicio 2.1 - Resumen general de pedidos

**Pregunta de negocio:** ¿Cuál es el resumen general de nuestros pedidos?

Calcula:
- Total de pedidos
- Pedidos por cada estado

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    order_status,
    COUNT(*) as cantidad,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM orders), 2) as porcentaje
FROM orders
GROUP BY order_status
ORDER BY cantidad DESC;
```
</details>

### Ejercicio 2.2 - Métricas de ventas

**Pregunta de negocio:** ¿Cuál es el valor total de ventas, el ticket promedio y el precio máximo?

Usa la tabla `order_items`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    COUNT(*) as total_items,
    ROUND(SUM(price), 2) as ingresos_totales,
    ROUND(AVG(price), 2) as ticket_promedio,
    MAX(price) as precio_maximo,
    MIN(price) as precio_minimo
FROM order_items;
```
</details>

### Ejercicio 2.3 - Métodos de pago preferidos

**Pregunta de negocio:** ¿Qué métodos de pago prefieren nuestros clientes?

Agrupa por `payment_type` y cuenta cuántas veces se usa cada uno:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    payment_type,
    COUNT(*) as cantidad_usos,
    ROUND(SUM(payment_value), 2) as valor_total,
    ROUND(AVG(payment_value), 2) as valor_promedio
FROM order_payments
GROUP BY payment_type
ORDER BY cantidad_usos DESC;
```
</details>

### Ejercicio 2.4 - Ventas por estado (ubicación)

**Pregunta de negocio:** ¿Cuáles son los 5 estados brasileños con más clientes?

Usa la tabla `customers`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    customer_state,
    COUNT(*) as total_clientes
FROM customers
GROUP BY customer_state
ORDER BY total_clientes DESC
LIMIT 5;
```
</details>

### Ejercicio 2.5 - Análisis de reviews

**Pregunta de negocio:** ¿Cuál es la distribución de las calificaciones de los clientes?

Agrupa por `review_score` de la tabla `order_reviews`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    review_score,
    COUNT(*) as cantidad,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM order_reviews), 2) as porcentaje
FROM order_reviews
GROUP BY review_score
ORDER BY review_score DESC;
```
</details>

---

## 🔹 PARTE 3: JOINs - Conectando Tablas

### Objetivo de Negocio
La información útil está distribuida en varias tablas. Necesitamos **combinarlas** para responder preguntas más complejas.

### Ejercicio 3.1 - Pedidos con información del cliente

**Pregunta de negocio:** ¿De qué ciudades provienen los pedidos más recientes?

Une `orders` con `customers` para ver la ciudad de cada pedido:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    o.order_id,
    o.order_purchase_timestamp,
    c.customer_city,
    c.customer_state
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
ORDER BY o.order_purchase_timestamp DESC
LIMIT 10;
```
</details>

### Ejercicio 3.2 - Ingresos por categoría de producto

**Pregunta de negocio:** ¿Cuáles son las categorías de productos que generan más ingresos?

Une `order_items` con `products` para obtener la categoría:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    p.product_category_name,
    COUNT(*) as items_vendidos,
    ROUND(SUM(oi.price), 2) as ingresos_totales,
    ROUND(AVG(oi.price), 2) as precio_promedio
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id
WHERE p.product_category_name IS NOT NULL
GROUP BY p.product_category_name
ORDER BY ingresos_totales DESC
LIMIT 10;
```
</details>

### Ejercicio 3.3 - Categorías en inglés

**Pregunta de negocio:** ¿Cómo se llaman las categorías top en inglés?

Une con la tabla `category_translation`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    ct.product_category_name_english as categoria,
    COUNT(*) as items_vendidos,
    ROUND(SUM(oi.price), 2) as ingresos_totales
FROM order_items oi
INNER JOIN products p ON oi.product_id = p.product_id
LEFT JOIN category_translation ct ON p.product_category_name = ct.product_category_name
WHERE p.product_category_name IS NOT NULL
GROUP BY ct.product_category_name_english
ORDER BY ingresos_totales DESC
LIMIT 10;
```
</details>

### Ejercicio 3.4 - Pedidos con reviews negativos

**Pregunta de negocio:** ¿Qué pedidos recibieron calificaciones de 1 o 2 estrellas y de qué estados son?

Une `orders`, `customers` y `order_reviews`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    c.customer_state,
    COUNT(*) as reviews_negativos,
    ROUND(AVG(r.review_score), 2) as score_promedio
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
INNER JOIN order_reviews r ON o.order_id = r.order_id
WHERE r.review_score <= 2
GROUP BY c.customer_state
ORDER BY reviews_negativos DESC
LIMIT 10;
```
</details>

### Ejercicio 3.5 - Análisis de vendedores

**Pregunta de negocio:** ¿Cuáles son los 5 vendedores con más ventas y de qué estados son?

Une `order_items` con `sellers`:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    s.seller_id,
    s.seller_city,
    s.seller_state,
    COUNT(*) as items_vendidos,
    ROUND(SUM(oi.price), 2) as ingresos_totales
FROM order_items oi
INNER JOIN sellers s ON oi.seller_id = s.seller_id
GROUP BY s.seller_id, s.seller_city, s.seller_state
ORDER BY ingresos_totales DESC
LIMIT 5;
```
</details>

---

## 🔹 PARTE 4: Subconsultas y Análisis Avanzado

### Objetivo de Negocio
Para análisis más sofisticados, necesitamos usar **subconsultas** y combinar múltiples operaciones.

### Ejercicio 4.1 - Pedidos sobre el promedio

**Pregunta de negocio:** ¿Cuántos pedidos tienen un valor superior al promedio?

Usa una subconsulta para calcular el promedio:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    COUNT(*) as pedidos_sobre_promedio,
    (SELECT ROUND(AVG(price), 2) FROM order_items) as precio_promedio
FROM order_items
WHERE price > (SELECT AVG(price) FROM order_items);
```
</details>

### Ejercicio 4.2 - Clientes VIP

**Pregunta de negocio:** ¿Quiénes son los clientes que han gastado más de R$1000 en total?

Une las tablas necesarias y filtra con HAVING:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    c.customer_id,
    c.customer_city,
    c.customer_state,
    COUNT(DISTINCT o.order_id) as total_pedidos,
    ROUND(SUM(oi.price), 2) as gasto_total
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_id, c.customer_city, c.customer_state
HAVING SUM(oi.price) > 1000
ORDER BY gasto_total DESC
LIMIT 10;
```
</details>

### Ejercicio 4.3 - Categorías con mejor review

**Pregunta de negocio:** ¿Cuáles son las 5 categorías de productos con mejor promedio de calificación?

(Solo considera categorías con al menos 100 reviews)

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    ct.product_category_name_english as categoria,
    COUNT(*) as total_reviews,
    ROUND(AVG(r.review_score), 2) as score_promedio
FROM order_reviews r
INNER JOIN orders o ON r.order_id = o.order_id
INNER JOIN order_items oi ON o.order_id = oi.order_id
INNER JOIN products p ON oi.product_id = p.product_id
LEFT JOIN category_translation ct ON p.product_category_name = ct.product_category_name
WHERE ct.product_category_name_english IS NOT NULL
GROUP BY ct.product_category_name_english
HAVING COUNT(*) >= 100
ORDER BY score_promedio DESC
LIMIT 5;
```
</details>

### Ejercicio 4.4 - Análisis temporal de ventas

**Pregunta de negocio:** ¿Cómo han evolucionado las ventas mes a mes?

Agrupa por año-mes las ventas:

In [ ]:
%%sql
-- Tu código aquí


<details>
<summary>💡 Ver solución</summary>

```sql
SELECT 
    strftime('%Y-%m', o.order_purchase_timestamp) as mes,
    COUNT(DISTINCT o.order_id) as total_pedidos,
    ROUND(SUM(oi.price), 2) as ingresos
FROM orders o
INNER JOIN order_items oi ON o.order_id = oi.order_id
WHERE o.order_status = 'delivered'
GROUP BY strftime('%Y-%m', o.order_purchase_timestamp)
ORDER BY mes;
```
</details>

---

## 🎯 DESAFÍO FINAL

### Reporte Ejecutivo

El CEO de Olist necesita un **reporte ejecutivo** con las siguientes métricas:

1. **Total de ingresos** (solo pedidos entregados)
2. **Ticket promedio** por pedido
3. **Top 3 categorías** por ingresos
4. **Top 3 estados** por número de clientes
5. **Calificación promedio** general

Crea las consultas necesarias para generar este reporte:

In [ ]:
%%sql
-- DESAFÍO: Tu reporte ejecutivo aquí


---

## 📝 Resumen de Conceptos SQL

| Concepto | Ejemplo | Uso |
|----------|---------|-----|
| SELECT | `SELECT columna FROM tabla` | Seleccionar datos |
| WHERE | `WHERE precio > 100` | Filtrar filas |
| ORDER BY | `ORDER BY fecha DESC` | Ordenar resultados |
| LIMIT | `LIMIT 10` | Limitar resultados |
| GROUP BY | `GROUP BY categoria` | Agrupar para agregar |
| HAVING | `HAVING COUNT(*) > 5` | Filtrar grupos |
| COUNT/SUM/AVG | `SUM(precio)` | Funciones de agregación |
| INNER JOIN | `JOIN tabla ON a.id = b.id` | Combinar tablas |
| LEFT JOIN | `LEFT JOIN tabla ON ...` | Incluir no coincidencias |
| Subconsulta | `WHERE x > (SELECT AVG...)` | Consultas anidadas |

---

**¡Felicitaciones!** Has completado el taller de SQL con datos reales de e-commerce. 🎉